# **micrograd advanced**

In [11]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import torch

## Value Class for Implementing the Backprop

In [8]:
class Value:

    def __init__(self,data,_children = (), _op = '',label = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda : None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data = {self.data})"

    #Adding Operations allowed by micrograd
    
    def __add__(self,otherVar):
        otherVar = otherVar if isinstance(otherVar,Value) else Value(otherVar)
        output = Value(self.data + otherVar.data, (self,otherVar),'+')
        
        def _backward():
            self.grad += 1.0 * output.grad
            otherVar.grad += 1.0 * output.grad
        output._backward = _backward

        return output

    def __radd__(self,otherVar): #otherVar + self
        return self + otherVar
    
    def __neg__(self):
        return self * -1

    def __sub__(self,otherVar):
        return self + (-otherVar)

    def __rsub__(self,otherVar): #OtherVar - self
        return otherVar + (-self)
    
    def __mul__(self,otherVar):
        otherVar = otherVar if isinstance(otherVar,Value) else Value(otherVar)
        output = Value(self.data * otherVar.data,(self,otherVar),'*')

        def _backward():
            self.grad += otherVar.data * output.grad
            otherVar.grad += self.data * output.grad  
        output._backward = _backward

        return output

    def __rmul__(self,otherVar):  #OtherVar * self
        return self * otherVar

    def __truediv__(self,otherVar): # self/otherVar
        return self * (otherVar ** -1)

    def __rtruediv__(self,otherVar):  #otherVar/self
        return (self ** -1) * (otherVar)
        
    def tanh(self):
        x = self.data
        t = ((math.exp(2*x)-1)/(math.exp(2*x)+1))
        output = Value(t,(self,),'tanh')
        
        def _backward():
            self.grad += (1 - (t**2)) * output.grad
        output._backward = _backward

        return output

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        output = Value(math.exp(x),(self,),'exp')

        def _backward():
            self.grad += output.data * output.grad # output.data is just math.exp(x)
        output._backward = _backward

        return output

    def __pow__(self,otherVar):
        assert isinstance(otherVar,(int,float)) # Ensuring only int or float powers are given
        output = Value(self.data**otherVar,(self,),f"**{otherVar}")

        def _backward():
            self.grad += otherVar * (self.data ** (otherVar - 1)) * output.grad
        output._backward = _backward

        return output

    def backward(self):
        self.grad = 1.0
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for node in reversed(topo):
            node._backward()

## Graph Visualization to demonstrate Backprop

In [13]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data,n.grad), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)
            
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

## Building a Neural Network / MLP(Multi Layer Perceptron)

In [26]:
class Neuron:

    def __init__(self,nin): #nin -> (n)umber of (in)puts
        #Creating nin Value objects as weights with random value between  -1 and 1
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]

        #Creating a bias for the neuron
        self.b = Value(random.uniform(-1,1))

    def __call__(self,x):
        # ∑w*x + b
        act = sum((wi*xi for wi,xi in zip(self.w,x)),self.b)
        output = act.tanh()
        return output

    def parameters(self):
        return self.w + [self.b]

class Layer:

    def __init__(self,nin,nout): #nout -> (n)umber of (out)puts
        #Creating nout number of neurons per layer
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self,x):
        #Initializing the neurons as done earlier
        outputs = [n(x) for n in self.neurons]
        return outputs[0] if len(outputs)==1 else outputs

    def parameters(self):
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params

class MLP:

    def __init__(self,nin,nouts): #nouts -> number of outputs per layer(is an array)
        #Adding input layer size also
        sz = [nin]+nouts
        
        #Creates layer by layer, taking sizes one by one
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]

    def __call__(self,x):
        #Calling these layers sequentially
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        params = []
        for layer in self.layers:
            ps = layer.parameters()
            params.extend(ps)
        return params

### Testing sample neural network

In [30]:
n = MLP(3,[4,4,1])
xs = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,-1.0,-1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] #Desired Targets

In [31]:
for k in range(50):

    #forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout-ygd)**2 for ygd, yout in zip(ys,ypred))

    #backward pass
    for p in n.parameters(): #Reset all the grad because they just keep accumulating due to +=
        p.grad = 0.0
    loss.backward()

    #gradient descent update
    for p in n.parameters():
        p.data += -0.05 * p.grad #Here 0.05 refers to the learning rate

    print(k," : ",loss.data)

0  :  4.698980092402184
1  :  1.7628668936542127
2  :  1.2374963251014068
3  :  0.8955803530286969
4  :  0.6508236250033467
5  :  0.48184210594239785
6  :  0.3688668755484664
7  :  0.2912765660213187
8  :  0.23608616786088926
9  :  0.19584241116783307
10  :  0.16576634097199153
11  :  0.14275097889904645
12  :  0.12475002593119984
13  :  0.11039055385491658
14  :  0.09873225755855913
15  :  0.08911756789013733
16  :  0.08107725551935266
17  :  0.0742700244090369
18  :  0.06844315966385892
19  :  0.06340641583142738
20  :  0.0590143730390872
21  :  0.05515430362831339
22  :  0.05173768729863738
23  :  0.04869418290685497
24  :  0.04596728133896126
25  :  0.04351112655387353
26  :  0.04128816029251818
27  :  0.03926735558010927
28  :  0.03742287660638046
29  :  0.035733051150994524
30  :  0.034179574748589125
31  :  0.032746888537657524
32  :  0.03142168860505424
33  :  0.030192535837466762
34  :  0.029049543285482968
35  :  0.027984123813480463
36  :  0.026988785011999517
37  :  0.02605

In [32]:
ypred

[Value(data = 0.9697747651024095),
 Value(data = -0.9174882605845308),
 Value(data = -0.9568109983797478),
 Value(data = 0.906925502030122)]